# PRS Tutorial

## Load in base data

In [1]:
import pandas as pd
# from: https://www.med.unc.edu/pgc/download-results/
filepath = '../files/daner_natgen_pgc_eas' # Dataset: scz2019asi, PMID: 31740837
base_df = pd.read_table(filepath, sep=' ') # load in file

### Filter base data

In [2]:
import basefilter

BASE_DATA = basefilter.Filter(base_df, \
                pval_range=(None, 0.05), \
                min_imputation_info_score=0.8, \
                    # https://choishingwan.github.io/PRS-Tutorial/base/#standard-gwas-qc
                    # https://www.nature.com/articles/s41596-020-0353-1
                remove_strand_ambig_snps=True)

10,694,924 SNPs in complete dataframe
-9,947,346 SNPs are out of the (None, 0.05) p-value bounds (747,578 SNPs remain)
-111,352 of the remaining SNPs are strand-ambiguous (636,226 SNPs remain)
-35,113 of the remaining SNPs are indels (601,113 SNPs remain)
-90,390 of the remaining SNPs have imputation scores lower than 0.8

510,723 SNPs remain


### Preview base data

In [3]:
BASE_DATA.FRAME.head()

,CHR,BP,A1,A2,FRQ_A_22778,FRQ_U_35362,INFO,OR,SE,P,ngt,HetISqt,HetDf,HetPVa,Nca,Nco,Neff
SNP,,,,,,,,,,,,,,,,,
rs190028338,1,100034461,A,C,0.9880,0.9810,0.954,1.62434,0.2165,0.025050,0,0.0,0.0,1.0000,1477.0,1680.0,1563.00
rs144918958,1,100072586,T,C,0.9740,0.9890,0.892,0.34249,0.3817,0.004997,0,0.0,0.0,1.0000,547.0,540.0,543.48
rs12136177,1,100149952,T,C,0.0654,0.0695,0.984,0.93128,0.0336,0.033930,6,45.3,3.0,0.1398,14583.0,17261.0,14916.61
rs6664416,1,100162963,A,C,0.0657,0.0698,0.989,0.93473,0.0334,0.043730,3,41.8,3.0,0.1608,14583.0,17261.0,14916.61
rs6686520,1,100167505,A,G,0.0656,0.0698,0.986,0.93379,0.0335,0.040620,1,38.2,3.0,0.1831,14583.0,17261.0,14916.61


## Set up ScoreEngine with base data

In [4]:
import scoring

engine = scoring.ScoreEngine(BASE_DATA.DICT)

### Calculate risk score at one SNP

In [5]:
from IPython.display import Image
Image(url= "./img/prs_eq.png", width=800, height=150)

In [6]:
engine.score_snp('rs189548203', 'AA')

-0.4468121715467305

In [7]:
engine.score_snp('rs189548203', 'AA', print=True)

rs189548203 (chr14:70775453)
Discov alleles:	TG (T=risk)
Target alleles:	TT (reversed? True)
Risk allele count: 2
Odds ratio: 0.8 => ln odds ratio: -0.223
Risk score: 2 * -0.223 = -0.4468



-0.4468121715467305

### Calculate PRS on random fictional sample

In [8]:
import random

alleles = ['A','G','T','C']

def random_index():
    """ Generate a random integer from 0 to 3. """
    return random.randint(0, len(alleles)-1)

def rand_alleles():
    """ Put together two random alleles from the list of possibilities. """
    return alleles[random_index()] + alleles[random_index()]

In [11]:
# generator of tuples composed of SNPs from base data and random alleles
count = len(BASE_DATA.DICT.keys())
pairings = zip(BASE_DATA.DICT.keys(), [rand_alleles() for _ in range(count)])

### Calculate the polygenic risk score of all SNP & allele pairings in `pairings` generator

In [12]:
engine.score_polygenic_risk(pairings, generator_count=count)

100%|██████████| 510723/510723 [00:02<00:00, 184101.24it/s]


620.5582032666241